# 使用脚本进行训练

除了🤗 Transformers笔记本外，还有示例脚本，演示如何使用PyTorch、TensorFlow或JAX/Flax为任务训练模型。

您还将找到我们在研究项目和遗留示例中使用的脚本，这些示例大多是由社区贡献的。这些脚本没有得到积极维护，并且需要一个特定版本的🤗 Transformers，这很可能与库的最新版本不兼容。

不应期望示例脚本可以直接在每个问题上运行，并且您可能需要根据您要解决的问题调整脚本。为了帮助您解决问题，大多数脚本完全暴露了数据的预处理方式，允许您根据需要进行编辑以适应您的用例。

对于您想要在示例脚本中实现的任何功能，请在提交Pull Request之前在论坛或问题中讨论。虽然我们欢迎错误修复，但我们不太可能合并以牺牲可读性为代价添加更多功能的Pull Request。

本指南将向您展示如何在PyTorch和TensorFlow中运行一个示例摘要训练脚本。除非另有说明，所有示例都预计可以在两个框架中运行。

## 设置
要成功运行最新版本的示例脚本，您必须在新的虚拟环境中从源代码安装🤗 Transformers：


```bash
git clone https://github.com/huggingface/transformers
cd transformers
pip install .
```


对于旧版本的示例脚本，请单击下面的切换按钮：

然后将您当前的🤗 Transformers克隆切换到特定版本，例如v3.5.1：


```bash
git checkout tags/v3.5.1
```


设置正确的库版本后，转到您选择的示例文件夹并安装示例特定要求：



```bash
pip install -r requirements.txt
```

## 运行脚本

示例脚本从🤗 Datasets库下载并预处理数据集。然后，脚本使用Keras在支持摘要的架构上微调数据集。以下示例展示了如何在CNN/DailyMail数据集上微调T5-small。由于T5模型的训练方式，该模型需要额外的source_prefix参数。这个提示让T5知道这是一个摘要任务。


```bash
python examples/tensorflow/summarization/run_summarization.py  \
    --model_name_or_path google-t5/t5-small \
    --dataset_name cnn_dailymail \
    --dataset_config "3.0.0" \
    --output_dir /tmp/tst-summarization  \
    --per_device_train_batch_size 8 \
    --per_device_eval_batch_size 16 \
    --num_train_epochs 3 \
    --do_train \
    --do_eval
```

## 分布式训练和混合精度

Trainer支持分布式训练和混合精度，这意味着您也可以在脚本中使用它。要启用这两个功能：

- 添加fp16参数以启用混合精度。
- 使用nproc_per_node参数设置要使用的GPU数量。


```bash
torchrun \
    --nproc_per_node 8 pytorch/summarization/run_summarization.py \
    --fp16 \
    --model_name_or_path google-t5/t5-small \
    --do_train \
    --do_eval \
    --dataset_name cnn_dailymail \
    --dataset_config "3.0.0" \
    --source_prefix "summarize: " \
    --output_dir /tmp/tst-summarization \
    --per_device_train_batch_size=4 \
    --per_device_eval_batch_size=4 \
    --overwrite_output_dir \
    --predict_with_generate
```


TensorFlow脚本利用MirroredStrategy进行分布式训练，您无需向训练脚本添加任何额外的参数。如果可用，默认情况下TensorFlow脚本将使用多个GPU。

## 在TPU上运行脚本

Tensor Processing Units (TPUs)专门设计用于加速性能。TensorFlow脚本利用TPUStrategy在TPU上进行训练。要使用TPU，请将TPU资源的名称传递给tpu参数。


```bash
python run_summarization.py  \
    --tpu name_of_tpu_resource \
    --model_name_or_path google-t5/t5-small \
    --dataset_name cnn_dailymail \
    --dataset_config "3.0.0" \
    --output_dir /tmp/tst-summarization  \
    --per_device_train_batch_size 8 \
    --per_device_eval_batch_size 16 \
    --num_train_epochs 3 \
    --do_train \
    --do_eval
```



## Run a script with 🤗 Accelerate

## 使用🤗 Accelerate运行脚本

🤗 Accelerate是一个仅适用于PyTorch的库，它提供了一种统一的方法，在多种设置（仅CPU、多个GPU、TPU）上训练模型，同时完全了解PyTorch训练循环。如果您尚未安装🤗 Accelerate，请确保安装它：

>"注意：由于Accelerate正在快速发展，必须安装加速脚本的git版本才能运行脚本"
```bash
pip install git+https://github.com/huggingface/accelerate
```


而不是使用run_summarization.py脚本，您需要使用run_summarization_no_trainer.py脚本。🤗 Accelerate支持的脚本将在文件夹中具有task_no_trainer.py文件。首先运行以下命令以创建并保存配置文件：


```bash
accelerate config
```

测试您的设置以确保配置正确：


```bash
accelerate test
```


现在您已经准备好启动训练：


```bash
accelerate launch run_summarization_no_trainer.py \
    --model_name_or_path google-t5/t5-small \
    --dataset_name cnn_dailymail \
    --dataset_config "3.0.0" \
    --source_prefix "summarize: " \
    --output_dir ~/tmp/tst-summarization
```




## Use a custom dataset


The summarization script supports custom datasets as long as they are a CSV or JSON Line file. When you use your own dataset, you need to specify several additional arguments:

- train_file and validation_file specify the path to your training and validation files.
- text_column is the input text to summarize.
- summary_column is the target text to output.
A summarization script using a custom dataset would look like this:

```bash
python examples/pytorch/summarization/run_summarization.py \
    --model_name_or_path google-t5/t5-small \
    --do_train \
    --do_eval \
    --train_file path_to_csv_or_jsonlines_file \
    --validation_file path_to_csv_or_jsonlines_file \
    --text_column text_column_name \
    --summary_column summary_column_name \
    --source_prefix "summarize: " \
    --output_dir /tmp/tst-summarization \
    --overwrite_output_dir \
    --per_device_train_batch_size=4 \
    --per_device_eval_batch_size=4 \
    --predict_with_generate
```



## 测试脚本

在承诺要完成可能需要几个小时的整个数据集之前，通常最好在较少的数据集示例上运行您的脚本，以确保一切按预期工作。使用以下参数将数据集截断为最大样本数：

- max_train_samples
- max_eval_samples
- max_predict_samples


```bash
python examples/pytorch/summarization/run_summarization.py \
    --model_name_or_path google-t5/t5-small \
    --max_train_samples 50 \
    --max_eval_samples 50 \
    --max_predict_samples 50 \
    --do_train \
    --do_eval \
    --dataset_name cnn_dailymail \
    --dataset_config "3.0.0" \
    --source_prefix "summarize: " \
    --output_dir /tmp/tst-summarization \
    --per_device_train_batch_size=4 \
    --per_device_eval_batch_size=4 \
    --overwrite_output_dir \
    --predict_with_generate
```


并非所有示例脚本都支持max_predict_samples参数。如果不确定您的脚本是否支持此参数，请添加-h参数进行检查：


```bash
examples/pytorch/summarization/run_summarization.py -h
```


## 从检查点恢复训练

另一个有用的选项是从先前的检查点恢复训练。这将确保您可以在中断训练后从上次中断的地方继续，而不必重新开始。有两种方法可以从检查点恢复训练。

第一种方法使用output_dir previous_output_dir参数从存储在output_dir中的最新检查点恢复训练。在这种情况下，您应该删除overwrite_output_dir：


```bash
python examples/pytorch/summarization/run_summarization.py
    --model_name_or_path google-t5/t5-small \
    --do_train \
    --do_eval \
    --dataset_name cnn_dailymail \
    --dataset_config "3.0.0" \
    --source_prefix "summarize: " \
    --output_dir /tmp/tst-summarization \
    --per_device_train_batch_size=4 \
    --per_device_eval_batch_size=4 \
    --output_dir previous_output_dir \
    --predict_with_generate
```


第二种方法使用resume_from_checkpoint path_to_specific_checkpoint参数从特定检查点文件夹恢复训练。


```bash
python examples/pytorch/summarization/run_summarization.py
    --model_name_or_path google-t5/t5-small \
    --do_train \
    --do_eval \
    --dataset_name cnn_dailymail \
    --dataset_config "3.0.0" \
    --source_prefix "summarize: " \
    --output_dir /tmp/tst-summarization \
    --per_device_train_batch_size=4 \
    --per_device_eval_batch_size=4 \
    --overwrite_output_dir \
    --resume_from_checkpoint path_to_specific_checkpoint \
    --predict_with_generate
```




## 分享您的模型

所有脚本都可以将您的最终模型上传到模型中心（Model Hub）。在开始之前，请确保您已登录到Hugging Face：


```bash
huggingface-cli login
```


然后在脚本中添加push_to_hub参数。此参数将创建一个具有您的Hugging Face用户名和output_dir中指定的文件夹名称的存储库。

要为您的存储库指定特定名称，请使用push_to_hub_model_id参数进行添加。该存储库将自动列在您的命名空间下。

以下示例显示了如何使用特定存储库名称上传模型：


```bash
python examples/pytorch/summarization/run_summarization.py
    --model_name_or_path google-t5/t5-small \
    --do_train \
    --do_eval \
    --dataset_name cnn_dailymail \
    --dataset_config "3.0.0" \
    --source_prefix "summarize: " \
    --push_to_hub \
    --push_to_hub_model_id finetuned-t5-cnn_dailymail \
    --output_dir /tmp/tst-summarization \
    --per_device_train_batch_size=4 \
    --per_device_eval_batch_size=4 \
    --overwrite_output_dir \
    --predict_with_generate
```